In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
#
import os
from dotenv import load_dotenv

In [8]:
load_dotenv()
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

In [9]:
loader = PyPDFLoader('ethics.pdf')
docs = loader.load()[:6]
#docs

In [10]:
len(docs)

6

In [11]:
docs[5].page_content

' \n171 \n 3.1. Факты нарушения Положения о  корпоративной этике работника ТПУ могут \nслужить основанием для отрицательного заключения кадровой  комиссии \nшколы (филиала), Кадровой аттестационной ко миссии Ученого совета ТПУ  \nв отношении претендента на замещение  должности профессорско -\nпреподавательского состава, конкурсной комиссии в отношении научного \nсостава.  \n3.2. Факты нарушения Положения о корпоративной этике работника ТПУ могут \nслужить основанием дл я отказа в поддержке университета при заявках  \nна гранты, участии в программах развития ТПУ, программах повышения \nквалификации, конференциях, привлечении к дополнительным работам в \nрамках НИР или ПОУ и любым зарубежным командировкам на срок от 1 года.  \n3.3. Считаются аморальным проступком, несовместимым с продолжением работы  \nпо данным должностям, следующие нарушения Положения о корпоративной \nэтике работниками из числа профессорско -преподавательского состава, \nнаучными работниками, педагогическими работника

In [12]:
#text_splitter = RecursiveCharacterTextSplitter(separators=['\n', '\n\n'], chunk_size=1000, chunk_overlap=20)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(docs)
#documents

In [13]:
len(documents)

17

In [14]:
documents[:5]

[Document(page_content='166 \n  \nПриложение № 19  \nк Коллективному договору ТПУ    \nна 2019 – 2021 годы  \n(продлён до 21.02.2025 г.)  \nПОЛОЖЕНИЕ  \nо корпоративной этике работника ТПУ  \n \n1. Общие положения корпоративной этики  \n \n1.1. Положение о корпоративной этике работника ТПУ распространяется на всех \nрабо тников Томского политехнического университета.  \n1.2. Поведение работника ТПУ на его рабочем месте служит развитию \nуниверситета. Должностные обязанности выполняются с единственной целью \n˗ повышение эффективности работы и защиты интересов университета.  \n1.3. Мы признае м, что наша цель ˗  работать не просто «хорошо», но «лучше \nвсех», а быстро и качественно выполненная  работа поощряется на всех \nуровнях.  \n1.4. Мы ведем свою деятельность таким образом, чтобы исключить возможность \nвозникновения конфликта интересов, когда прямые или косвенные личные \nинтересы работника любого уровня несовместимы с интересами ТПУ  \nили нацелены на получение личной выгоды с и

### Заменить OpenAIEmbeddings

In [4]:
'''
from langchain_community.embeddings import HuggingFaceEmbeddings

model_id = 'sentence-transformers/all-MiniLM-L6-v2'
#model_kwargs = {'device': 'cuda'}
model_kwargs = {'device': 'cpu'}

huggingface_embeddings = HuggingFaceEmbeddings(
    model_name=model_id,
    model_kwargs=model_kwargs
)
'''
pass

In [5]:
#model_id = 'sentence-transformers/all-MiniLM-l6-v2'
model_id = 'BAAI/bge-small-en-v1.5'

#model_args = {'device': 'cuda'}
model_args = {'device': 'cpu'}

huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_id,
    model_kwargs=model_args,
    encode_kwargs={'normalize_embeddings': True}
)

e:\Progi\PythonProg\NLP\LangChain\KrishNaik\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
e:\Progi\PythonProg\NLP\LangChain\KrishNaik\.venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Андрей\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode

In [15]:
query = 'выставление завышенных требований к экзаменам'

In [16]:
db_faiss = FAISS.from_documents(documents, embedding=huggingface_embeddings)
db_faiss

In [17]:
retireved_results = db_faiss.similarity_search(query)
#retireved_results
print(retireved_results[0].page_content)

169 
 2.10. Выставление завышенных требований к экзаменам и зачетам, так же как и 
явно заниженные требования и формализм в проставлении оценок, не 
являются поощряемыми формами поведения в Томском политехническом 
университете.  
2.11. Любая информация, предоставляемая обучающимся для подготовки  
к экзаменам и зачетам, должна носить исчерпывающий  характер, быть 
доступной в бумажном и электронном виде, но не содержать готовых ответов.  
2.12. Преподаватель оценивает обучающихся независимо и объективно, 
основываясь на качестве их ответов, но не на свойствах личности и своих 
личных предпочтениях. Снижение и ли, напротив, повышение требований в 
ходе испытаний, по сравнению с ранее заявленными, не поощряется в рамках 
настоящего Кодекса.  
2.13. Томский политехнический университет придерживается высоких стандартов 
при проведении научных исследований и представлении их результатов. 
Допускаются к публикации только оригинальные научные труды, содержащие


In [16]:
#for i in range(0, len(retireved_results)): print(retireved_results[i].page_content)

In [18]:
llm = Ollama(model='sutyrin/saiga_mistral_7b')
llm

Ollama(model='sutyrin/saiga_mistral_7b')

In [19]:
prompt_str = """
Нужно ответить на вопрос, основываясь только на предоставленном контексте.
Подумай шаг за шагом, прежде чем предоставить подробный ответ.
<context>
{context}
</context>
Questions: {input}
"""

prompt = ChatPromptTemplate.from_template(prompt_str)

In [20]:
retriever = db_faiss.as_retriever()

document_chain = create_stuff_documents_chain(llm, prompt)

'''
Цепочка поиска: эта цепочка принимает запрос пользователя,
который передается извлекателю для извлечения релевантных документов.
Эти документы (и исходные входные данные) затем передаются в LLM для генерации ответа
'''
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [21]:
# Иммитация запроса от пользователя
response = retrieval_chain.invoke({'input': 'выставление завышенных требований к экзаменам'})
response

{'input': 'выставление завышенных требований к экзаменам',
 'context': [Document(page_content='169 \n 2.10. Выставление завышенных требований к экзаменам и зачетам, так же как и \nявно заниженные требования и формализм в проставлении оценок, не \nявляются поощряемыми формами поведения в Томском политехническом \nуниверситете.  \n2.11. Любая информация, предоставляемая обучающимся для подготовки  \nк экзаменам и зачетам, должна носить исчерпывающий  характер, быть \nдоступной в бумажном и электронном виде, но не содержать готовых ответов.  \n2.12. Преподаватель оценивает обучающихся независимо и объективно, \nосновываясь на качестве их ответов, но не на свойствах личности и своих \nличных предпочтениях. Снижение и ли, напротив, повышение требований в \nходе испытаний, по сравнению с ранее заявленными, не поощряется в рамках \nнастоящего Кодекса.  \n2.13. Томский политехнический университет придерживается высоких стандартов \nпри проведении научных исследований и представлении их результ

In [22]:
print(response['answer'])

Выставление завышенных требований к экзаменам и зачетам является непоощряемым формами поведения в Томском политехническом университете. Это противоречит принципам честности, объективности и беспристрастности, которые должны соблюдаться при оценивании студентов.
